# MLOps: Guía Completa con Ejemplos

## 1. Introducción a MLOps

```markdown
# Introducción a MLOps

Machine Learning Operations (MLOps) es un conjunto de prácticas que automatizan y simplifican los flujos de trabajo y despliegues de machine learning. Representa una cultura y práctica que unifica el desarrollo de aplicaciones ML (Dev) con el despliegue y operaciones de sistemas ML (Ops).

El mercado de MLOps se proyecta a alcanzar los 10.4 mil millones de USD para 2028, creciendo a un impresionante CAGR del 28.6% desde 2022 hasta 2028. Aproximadamente el 80% de las organizaciones planean adoptar MLOps en los próximos dos años.
```

## 2. Componentes Principales de MLOps

```markdown
## Componentes Principales de MLOps

| Componente | Descripción | Herramientas/Frameworks | Usos |
|------------|-------------|-------------------------|------|
| **Version Control** | Control de versiones que se extiende más allá del código para incluir datos y modelos | - Git (código)<br>- DVC (Data Version Control)<br>- lakeFS<br>- Pachyderm | - Trazabilidad de cambios<br>- Reproducibilidad<br>- Capacidad de rollback<br>- Colaboración eficiente |
| **CI/CD** | Pipelines que automatizan pruebas y despliegue de modelos ML | - GitHub Actions<br>- GitLab CI/CD<br>- Jenkins<br>- CircleCI<br>- Azure Pipelines | - Validación y pruebas automatizadas<br>- Procesos de despliegue consistentes<br>- Ciclos de iteración más rápidos<br>- Reducción de errores humanos |
| **Orchestration** | Coordinación y automatización de pasos en flujos de trabajo ML | - Airflow<br>- Prefect<br>- Dagster<br>- Luigi<br>- Metaflow<br>- Flyte<br>- Kubeflow Pipelines | - Programación y ejecución de componentes<br>- Gestión de dependencias<br>- Manejo de fallos y reintentos<br>- Monitoreo de ejecución |
| **Compute** | Gestión de infraestructura para entrenamiento y servicio de modelos | - AWS<br>- Google Cloud<br>- Azure<br>- Servicios especializados de ML | - Entornos de entrenamiento distribuido para GPUs<br>- Recursos de cómputo escalables<br>- Optimización de costos<br>- Despliegues en nube híbrida |
| **Serving** | Despliegue de modelos entrenados en producción | - TorchServe<br>- Seldon Core<br>- AWS SageMaker<br>- Contenedores Docker | - Despliegue containerizado<br>- Endpoints para inferencia en tiempo real y batch<br>- Gestión de API<br>- Manejo de versiones |
| **Monitoring** | Sistemas para seguimiento de modelos desplegados | - Evidently AI<br>- Arize AI<br>- Fiddler<br>- Prometheus<br>- Grafana | - Detección de model drift y data drift<br>- Seguimiento de rendimiento<br>- Alertas<br>- Monitoreo de métricas del sistema |
| **Metrics Registry** | Catálogo de modelos ML y sus versiones | - MLflow<br>- Weights & Biases<br>- DVC Studio | - Seguimiento de linaje y procedencia<br>- Registro de métricas de rendimiento<br>- Descubrimiento y compartición de modelos<br>- Promoción por etapas (dev, test, prod) |
```

## 3. Ejemplos de Código por Componente

### 3.1 Version Control - DVC

```python
# Ejemplo de Version Control con DVC
# ---------------------------------
# Instalación: pip install dvc

# Inicializar DVC en un proyecto
!dvc init

# Agregar un conjunto de datos grande a DVC
!dvc add data/large_dataset.csv

# Añadir información de seguimiento a Git
!git add data/large_dataset.csv.dvc .gitignore

# Hacer commit de cambios
!git commit -m "Añadir dataset grande mediante DVC"

# Ejemplo básico de uso de DVC en un flujo de trabajo
import os

# Crear un script simple de procesamiento de datos
with open('process_data.py', 'w') as f:
    f.write("""
import pandas as pd
import numpy as np

# Cargar datos
data = pd.read_csv('data/large_dataset.csv')

# Procesar datos
processed_data = data.dropna()
processed_data['nueva_columna'] = np.log(processed_data['columna_existente'] + 1)

# Guardar datos procesados
processed_data.to_csv('data/processed_dataset.csv', index=False)
""")

# Crear un archivo de etapa DVC
with open('dvc.yaml', 'w') as f:
    f.write("""
stages:
  process:
    cmd: python process_data.py
    deps:
      - process_data.py
      - data/large_dataset.csv
    outs:
      - data/processed_dataset.csv
""")

# Ejecutar la etapa
!dvc repro

# Ver el gráfico de dependencias
!dvc dag
```

### 3.2 CI/CD - GitHub Actions

```python
# Ejemplo de archivo YAML para GitHub Actions
# Guarda esto como .github/workflows/ml-pipeline.yml

ci_cd_example = """
name: ML Model CI/CD

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  test-and-deploy:
    runs-on: ubuntu-latest
    steps:
    - uses: actions/checkout@v3
    - name: Set up Python
      uses: actions/setup-python@v4
      with:
        python-version: '3.10'
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install pytest scikit-learn pandas numpy mlflow
        if [ -f requirements.txt ]; then pip install -r requirements.txt; fi
    
    - name: Test with pytest
      run: |
        pytest tests/
    
    - name: Train model
      run: |
        python train_model.py
    
    - name: Evaluate model
      run: |
        python evaluate_model.py
        
    - name: Deploy model (if tests pass and on main)
      if: success() && github.ref == 'refs/heads/main'
      run: |
        python deploy_model.py
"""

print(ci_cd_example)

# Ejemplo simple de script de entrenamiento
with open('train_model.py', 'w') as f:
    f.write("""
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pickle

# Generar datos de ejemplo
X = np.random.rand(1000, 10)
y = (X[:, 0] + X[:, 1] > 1).astype(int)

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Guardar modelo
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Modelo entrenado y guardado con éxito")
""")
```

### 3.3 Orchestration - Airflow

```python
# Ejemplo de DAG en Apache Airflow
# -----------------------------

# Instalar Airflow: pip install apache-airflow

# Ejemplo de definición de DAG
dag_example = """
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.python import PythonOperator
from airflow.sensors.filesystem import FileSensor

# Funciones para las tareas
def extract_data():
    print("Extrayendo datos...")
    # Código para extraer datos de la fuente
    with open('/tmp/data_extracted.txt', 'w') as f:
        f.write('Datos extraídos')

def transform_data():
    print("Transformando datos...")
    # Código para transformar datos
    with open('/tmp/data_transformed.txt', 'w') as f:
        f.write('Datos transformados')

def train_model():
    print("Entrenando modelo...")
    # Código para entrenar modelo
    with open('/tmp/model_trained.txt', 'w') as f:
        f.write('Modelo entrenado')

def evaluate_model():
    print("Evaluando modelo...")
    # Código para evaluar modelo
    with open('/tmp/model_evaluated.txt', 'w') as f:
        f.write('Modelo evaluado')

def deploy_model():
    print("Desplegando modelo...")
    # Código para desplegar modelo
    with open('/tmp/model_deployed.txt', 'w') as f:
        f.write('Modelo desplegado')

# Configuración del DAG
default_args = {
    'owner': 'mlops_team',
    'depends_on_past': False,
    'email': ['mlops@example.com'],
    'email_on_failure': True,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

with DAG(
    'ml_training_pipeline',
    default_args=default_args,
    description='Pipeline de entrenamiento de ML con Airflow',
    schedule_interval=timedelta(days=1),
    start_date=datetime(2025, 1, 1),
    catchup=False,
    tags=['ml', 'training'],
) as dag:
    
    # Tareas
    extract_task = PythonOperator(
        task_id='extract_data',
        python_callable=extract_data,
    )
    
    # Sensor para esperar que los datos estén disponibles
    wait_for_data = FileSensor(
        task_id='wait_for_data',
        filepath='/tmp/data_extracted.txt',
        poke_interval=30,  # Revisar cada 30 segundos
        timeout=60 * 5,  # Tiempo límite de 5 minutos
    )
    
    transform_task = PythonOperator(
        task_id='transform_data',
        python_callable=transform_data,
    )
    
    train_task = PythonOperator(
        task_id='train_model',
        python_callable=train_model,
    )
    
    evaluate_task = PythonOperator(
        task_id='evaluate_model',
        python_callable=evaluate_model,
    )
    
    deploy_task = PythonOperator(
        task_id='deploy_model',
        python_callable=deploy_model,
    )
    
    # Definir el flujo de ejecución
    extract_task >> wait_for_data >> transform_task >> train_task >> evaluate_task >> deploy_task
"""

print(dag_example)
```

### 3.4 Compute - Ejemplo de configuración con AWS

```python
# Ejemplo de configuración de infraestructura AWS para ML con Boto3
# -----------------------------------------------------------------
# Instalación: pip install boto3

import boto3
import json

# Ejemplo básico de configuración de instancia EC2 para ML
def create_ml_ec2_instance():
    ec2 = boto3.resource('ec2')
    
    # Crear una instancia con GPU para entrenamiento
    instances = ec2.create_instances(
        ImageId='ami-0c55b159cbfafe1f0',           # AMI con software ML preinstalado
        InstanceType='p3.2xlarge',                  # Instancia con GPU para ML
        MinCount=1,
        MaxCount=1,
        KeyName='my-key-pair',                      # Tu key pair
        SecurityGroupIds=['sg-12345678'],           # Tu security group
        BlockDeviceMappings=[
            {
                'DeviceName': '/dev/sda1',
                'Ebs': {
                    'VolumeSize': 100,              # 100GB para datasets y modelos
                    'DeleteOnTermination': True,
                    'VolumeType': 'gp2',
                }
            },
        ],
        TagSpecifications=[
            {
                'ResourceType': 'instance',
                'Tags': [
                    {
                        'Key': 'Name',
                        'Value': 'ML-Training-Instance'
                    },
                    {
                        'Key': 'Project',
                        'Value': 'Customer-Churn-Prediction'
                    }
                ]
            }
        ],
        UserData='''#!/bin/bash
                    pip install -U scikit-learn tensorflow pytorch
                    git clone https://github.com/company/ml-project.git
                    cd ml-project
                    python setup.py install
                 '''
    )
    
    print(f"Instancia creada con ID: {instances[0].id}")
    return instances[0].id

# Ejemplo de configuración de SageMaker para entrenamiento
def create_sagemaker_training_job():
    sagemaker = boto3.client('sagemaker')
    
    training_job_name = 'churn-prediction-xgboost-' + str(int(time.time()))
    
    # Configurar un trabajo de entrenamiento de SageMaker
    response = sagemaker.create_training_job(
        TrainingJobName=training_job_name,
        AlgorithmSpecification={
            'TrainingImage': '123456789012.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
            'TrainingInputMode': 'File'
        },
        RoleArn='arn:aws:iam::123456789012:role/SageMakerRole',
        InputDataConfig=[
            {
                'ChannelName': 'train',
                'DataSource': {
                    'S3DataSource': {
                        'S3DataType': 'S3Prefix',
                        'S3Uri': 's3://my-bucket/training-data/',
                        'S3DataDistributionType': 'FullyReplicated'
                    }
                },
                'ContentType': 'csv',
                'CompressionType': 'None'
            }
        ],
        OutputDataConfig={
            'S3OutputPath': 's3://my-bucket/output/'
        },
        ResourceConfig={
            'InstanceType': 'ml.m5.xlarge',
            'InstanceCount': 1,
            'VolumeSizeInGB': 50
        },
        StoppingCondition={
            'MaxRuntimeInSeconds': 86400
        },
        HyperParameters={
            'max_depth': '6',
            'eta': '0.2',
            'objective': 'binary:logistic',
            'num_round': '100'
        },
        Tags=[
            {
                'Key': 'Project',
                'Value': 'CustomerChurn'
            }
        ]
    )
    
    print(f"Trabajo de entrenamiento creado: {training_job_name}")
    return response

# Estos son ejemplos y requerirían credenciales AWS configuradas para ejecutarse
# print("Ejemplos de configuración AWS para ML")
```

### 3.5 Model Serving - Flask API para servir un modelo

```python
# Ejemplo de Servicio de Modelo con Flask
# --------------------------------------
# Instalación: pip install flask scikit-learn pandas

import pickle
import numpy as np
from flask import Flask, request, jsonify

# Crear una aplicación Flask
app = Flask(__name__)

# Cargar un modelo pre-entrenado (simulado aquí)
def load_model():
    # En un caso real, cargarías un modelo previamente entrenado
    from sklearn.ensemble import RandomForestClassifier
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    # Entrenamiento simulado
    X = np.random.rand(1000, 4)
    y = (X[:, 0] + X[:, 1] > 1).astype(int)
    model.fit(X, y)
    return model

# Cargar el modelo al iniciar
model = load_model()

@app.route('/predict', methods=['POST'])
def predict():
    """Endpoint para realizar predicciones."""
    # Obtener datos de la solicitud
    data = request.json
    
    # Verificar que los datos tengan el formato esperado
    if not data or 'features' not in data:
        return jsonify({'error': 'No features provided'}), 400
    
    try:
        # Convertir datos a numpy array
        features = np.array(data['features'])
        
        # Hacer predicción
        prediction = model.predict(features.reshape(1, -1))
        prediction_proba = model.predict_proba(features.reshape(1, -1)).tolist()
        
        # Devolver respuesta
        return jsonify({
            'prediction': int(prediction[0]),
            'probability': prediction_proba[0]
        })
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/health', methods=['GET'])
def health():
    """Endpoint para verificar la salud del servicio."""
    return jsonify({'status': 'healthy'})

# Código para ejecutar la aplicación
if __name__ == '__main__':
    print("Iniciando servidor de predicción en http://localhost:5000")
    print("Endpoints disponibles:")
    print("  - POST /predict - Para realizar predicciones")
    print("  - GET /health - Para verificar la salud del servicio")
    # app.run(host='0.0.0.0', port=5000)
    
# Ejemplo de solicitud:
example_request = {
    "features": [0.5, 0.8, 0.2, 0.1]
}
print(f"\nEjemplo de solicitud: {example_request}")
```

### 3.6 Monitoring - Monitoreo de Modelos con Evidently

```python
# Ejemplo de Monitoreo de Modelos con Evidently
# -------------------------------------------
# Instalación: pip install evidently pandas scikit-learn

import pandas as pd
import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, TargetDriftPreset, RegressionPreset

# Generar datos de ejemplo
def generate_data():
    # Cargar datos de diabetes
    diabetes = load_diabetes()
    X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
    y = diabetes.target
    
    # Dividir en train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Entrenar un modelo
    model = RandomForestRegressor(random_state=42)
    model.fit(X_train, y_train)
    
    # Crear dataset de referencia (training)
    reference_data = X_train.copy()
    reference_data['target'] = y_train
    
    # Crear dataset actual (test)
    current_data = X_test.copy()
    current_data['target'] = y_test
    
    # Agregar predicciones
    reference_data['prediction'] = model.predict(X_train)
    current_data['prediction'] = model.predict(X_test)
    
    return reference_data, current_data

# Crear un informe de monitoreo
def create_monitoring_report(reference_data, current_data):
    # Reporte de Data Drift
    data_drift_report = Report(metrics=[DataDriftPreset()])
    data_drift_report.run(reference_data=reference_data, current_data=current_data)
    data_drift_json = data_drift_report.json()
    
    # Reporte de Target Drift
    target_drift_report = Report(metrics=[TargetDriftPreset()])
    target_drift_report.run(reference_data=reference_data, current_data=current_data)
    target_drift_json = target_drift_report.json()
    
    # Reporte de Rendimiento de Regresión
    regression_report = Report(metrics=[RegressionPreset()])
    regression_report.run(reference_data=reference_data, current_data=current_data,
                          column_mapping={'target': 'target', 'prediction': 'prediction'})
    regression_json = regression_report.json()
    
    return {
        'data_drift': data_drift_json,
        'target_drift': target_drift_json,
        'regression_performance': regression_json
    }

# Generar datos y crear informe
reference_data, current_data = generate_data()
report = create_monitoring_report(reference_data, current_data)

print("Informe de monitoreo generado. El informe contiene métricas de:")
print("1. Data Drift - Detecta cambios en la distribución de las features")
print("2. Target Drift - Detecta cambios en la distribución del target")
print("3. Regression Performance - Evalúa el rendimiento del modelo de regresión")

# En un caso real, guardarías estos informes o los enviarías a un dashboard
```

### 3.7 Metrics Registry - MLflow

```python
# Ejemplo de Metrics Registry con MLflow
# -----------------------------------
# Instalación: pip install mlflow scikit-learn pandas matplotlib

import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configurar el servidor MLflow
# En producción, configurarías un servidor centralizado
# mlflow.set_tracking_uri("http://localhost:5000")

# Crear un experimento
mlflow.set_experiment("customer_churn_prediction")

# Generar datos de ejemplo
def generate_sample_data():
    # Datos simulados de clientes
    n_samples = 1000
    X = np.random.rand(n_samples, 5)  # 5 características
    # Crear una relación para y basada en las características
    y = (X[:, 0] > 0.7) | ((X[:, 1] > 0.5) & (X[:, 2] < 0.3))
    y = y.astype(int)  # Convertir a 0 y 1
    
    # Convertir a DataFrame para mejor manejo
    feature_names = ['usage_time', 'num_complaints', 'subscription_length', 'age', 'num_services']
    X_df = pd.DataFrame(X, columns=feature_names)
    
    return X_df, y

# Entrenar y registrar modelos
def train_and_log_model():
    # Generar datos
    X, y = generate_sample_data()
    
    # Dividir en training y test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Iniciar el run de MLflow
    with mlflow.start_run(run_name="rf_model_run") as run:
        # Registrar parámetros
        params = {
            "n_estimators": 100,
            "max_depth": 5,
            "min_samples_split": 2,
            "random_state": 42
        }
        mlflow.log_params(params)
        
        # Entrenar modelo
        rf = RandomForestClassifier(**params)
        rf.fit(X_train, y_train)
        
        # Realizar predicciones
        y_pred = rf.predict(X_test)
        
        # Calcular métricas
        metrics = {
            "accuracy": accuracy_score(y_test, y_pred),
            "precision": precision_score(y_test, y_pred),
            "recall": recall_score(y_test, y_pred),
            "f1": f1_score(y_test, y_pred)
        }
        
        # Registrar métricas
        mlflow.log_metrics(metrics)
        
        # Registrar modelo
        mlflow.sklearn.log_model(rf, "random_forest_model",
                                registered_model_name="churn_prediction_model")
        
        # Registrar feature importance como una figura
        feature_importance = pd.DataFrame(
            rf.feature_importances_,
            index=X.columns,
            columns=['importance']
        ).sort_values('importance', ascending=False)
        
        # En un caso real, aquí generarías y guardarías una figura
        # fig = feature_importance.plot(kind='bar', figsize=(10, 6)).get_figure()
        # fig.savefig("feature_importance.png")
        # mlflow.log_artifact("feature_importance.png")
        
        # Registrar ejemplo de datos de entrada
        mlflow.log_input(X_train, context="training")
        
        print(f"Modelo entrenado y registrado con MLflow - Run ID: {run.info.run_id}")
        print(f"Métricas: {metrics}")
        print(f"El modelo ha sido registrado como 'churn_prediction_model'")
        
        return run.info.run_id, metrics

# Ejecutar el entrenamiento y registro
run_id, metrics = train_and_log_model()

# Mostrar cómo cargar un modelo registrado
print("\nPara cargar el modelo registrado:")
print("model = mlflow.sklearn.load_model('models:/churn_prediction_model/latest')")

# Mostrar cómo usar el modelo para predicciones
print("\nEjemplo de uso para predicciones:")
print("""
import mlflow.sklearn
import pandas as pd

# Cargar el modelo registrado
model = mlflow.sklearn.load_model('models:/churn_prediction_model/latest')

# Preparar datos para predicción
new_data = pd.DataFrame({
    'usage_time': [0.8],
    'num_complaints': [0.2],
    'subscription_length': [0.6],
    'age': [0.4],
    'num_services': [0.7]
})

# Hacer predicción
prediction = model.predict(new_data)
print(f'Predicción: {prediction}')
""")
```

## 4. Flujo de Trabajo de MLOps

```markdown
## Flujo de Trabajo de MLOps

| Etapa | Descripción | Herramientas Destacadas | Mejores Prácticas |
|-------|-------------|-------------------------|-------------------|
| **Ingesta de Datos** | Recopilación y validación de datos | - Feast<br>- Tecton<br>- Hopsworks | - Implementar verificaciones de calidad<br>- Aplicar versionado<br>- Establecer políticas de gobernanza |
| **Preparación de Datos** | Limpieza, transformación y feature engineering | - Pandas<br>- PySpark<br>- Feature Stores | - Estandarizar pipelines de datos<br>- Automatizar transformaciones<br>- Documentar metadatos |
| **Entrenamiento de Modelos** | Desarrollo y ajuste de modelos | - PyTorch<br>- TensorFlow<br>- Scikit-learn<br>- MLflow | - Implementar validación cruzada<br>- Aplicar CI/CD para entrenamiento<br>- Mantener reproducibilidad |
| **Evaluación de Modelos** | Verificación del rendimiento del modelo | - MLflow<br>- Weights & Biases<br>- Neptune.ai | - Definir métricas claras<br>- Comparar con líneas base<br>- Validar con datos representativos |
| **Registro de Modelos** | Catalogación y control de versiones de modelos | - MLflow Model Registry<br>- DVC Studio | - Mantener metadatos completos<br>- Implementar políticas de aprobación<br>- Establecer etiquetas de versión |
| **Despliegue** | Puesta en producción de modelos | - Docker<br>- Kubernetes<br>- Seldon Core<br>- BentoML | - Usar contenedores<br>- Implementar despliegues blue-green<br>- Automatizar pruebas de integración |
| **Monitoreo** | Seguimiento del rendimiento y alertas | - Evidently AI<br>- Prometheus<br>- Grafana | - Monitorear drift del modelo<br>- Establecer alertas proactivas<br>- Implementar paneles de control |
| **Reentrenamiento** | Actualización periódica de modelos | - Airflow<br>- Kubeflow<br>- MLflow | - Definir disparadores de reentrenamiento<br>- Automatizar evaluación/despliegue<br>- Mantener historial de rendimiento |
```

## 5. Tendencias Emergentes en MLOps

```markdown
## Tendencias Emergentes en MLOps

| Tendencia | Descripción | Tecnologías Relacionadas |
|-----------|-------------|--------------------------|
| **LLMOps** | Prácticas especializadas para modelos de lenguaje grandes | - Frameworks de fine-tuning<br>- Herramientas de prompt engineering<br>- Sistemas RAG |
| **Edge MLOps** | Despliegue y gestión de modelos en dispositivos edge | - TensorFlow Lite<br>- ONNX Runtime<br>- Edge ML frameworks |
| **MLOps Automatizado** | Mayor automatización de flujos de trabajo ML | - AutoML<br>- Hyperparameter tuning automatizado<br>- Feature engineering automatizado |
| **MLOps para IA Responsable** | Integración de consideraciones éticas en flujos de trabajo | - Fairness indicators<br>- Herramientas de explicabilidad<br>- Frameworks de cumplimiento |
| **Prácticas de IA Sostenible** | Enfoque en eficiencia energética y sostenibilidad | - Técnicas de entrenamiento eficientes<br>- Optimización de modelos<br>- Métricas de huella de carbono |
```

### 5.1 Ejemplo de LLMOps - Fine-tuning un modelo para RAG

```python
# Ejemplo de LLMOps - Retrieval Augmented Generation (RAG)
# --------------------------------------------------------
# Instalación: pip install langchain openai

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

## Componentes Principales de MLOps

| Componente | Descripción | Herramientas/Frameworks | Usos |
|------------|-------------|-------------------------|------|
| **Version Control** | Control de versiones que se extiende más allá del código para incluir datos y modelos | - Git (código)<br>- DVC (Data Version Control)<br>- lakeFS<br>- Pachyderm | - Trazabilidad de cambios<br>- Reproducibilidad<br>- Capacidad de rollback<br>- Colaboración eficiente |
| **CI/CD** | Pipelines que automatizan pruebas y despliegue de modelos ML | - GitHub Actions<br>- GitLab CI/CD<br>- Jenkins<br>- CircleCI<br>- Azure Pipelines | - Validación y pruebas automatizadas<br>- Procesos de despliegue consistentes<br>- Ciclos de iteración más rápidos<br>- Reducción de errores humanos |
| **Orchestration** | Coordinación y automatización de pasos en flujos de trabajo ML | - Airflow<br>- Prefect<br>- Dagster<br>- Luigi<br>- Metaflow<br>- Flyte<br>- Kubeflow Pipelines | - Programación y ejecución de componentes<br>- Gestión de dependencias<br>- Manejo de fallos y reintentos<br>- Monitoreo de ejecución |
| **Compute** | Gestión de infraestructura para entrenamiento y servicio de modelos | - AWS<br>- Google Cloud<br>- Azure<br>- Servicios especializados de ML | - Entornos de entrenamiento distribuido para GPUs<br>- Recursos de cómputo escalables<br>- Optimización de costos<br>- Despliegues en nube híbrida |
| **Serving** | Despliegue de modelos entrenados en producción | - TorchServe<br>- Seldon Core<br>- AWS SageMaker<br>- Contenedores Docker | - Despliegue containerizado<br>- Endpoints para inferencia en tiempo real y batch<br>- Gestión de API<br>- Manejo de versiones |
| **Monitoring** | Sistemas para seguimiento de modelos desplegados | - Evidently AI<br>- Arize AI<br>- Fiddler<br>- Prometheus<br>- Grafana | - Detección de model drift y data drift<br>- Seguimiento de rendimiento<br>- Alertas<br>- Monitoreo de métricas del sistema |
| **Metrics Registry** | Catálogo de modelos ML y sus versiones | - MLflow<br>- Weights & Biases<br>- DVC Studio | - Seguimiento de linaje y procedencia<br>- Registro de métricas de rendimiento<br>- Descubrimiento y compartición de modelos<br>- Promoción por etapas (dev, test, prod) |